In [1]:
import numpy as np
import pandas as pd
import openai
import requests
from stockfish import Stockfish
import time
import re
import chess
from tqdm import tqdm
import os


In [2]:
# used to connect to OpenAI API
# please insert your OpenAI security key
api_secret_key = "#######################"

# initialize the OpenAI API key
openai.api_key = api_secret_key

## Approach : 

<ol> 

<li> Pass a starting position to ChatGPT (in Algebraic Notation : 1e4 1..e5) </li>
<li> Obtain a move from ChatGPT (repeat query if the answer is an illegal move) </li>
<li> Give the move to a chess engine (Stockfish) - checks if the move is a top move and assigns scores </li>
<li> Instantiate a chess board and perform the move indicated by ChatGPT </li>
<li> Repeat
</ol>


In [3]:
def chatgpt_generate_next_move(moves, player):
    """
    Given a string encoding a sequence of moves in the format : 1.e4 1...e5 2. Nf3,
    return the answer given by ChatGPT to the question : 'What is the best next move?'

    Parameters
    ----------
    moves : string
        encodes a succession of chess moves in one of the follwing formats:
        FEN (Forsyth–Edwards Notation) - e.g. 'rnbqkbnr/pp1ppppp/8/2p5/4P3/8/PPPP1PPP/RNBQKBNR w KQkq c6 0 2'
        Standard Algebraic Notation ()
        SAN (Standard Algebraic Notation)
        LAN (Long Algebraic Notation)
    """

    # generate text using the GPT-3 model
    model_engine = "text-davinci-002"

    prompt = f"What is the best move for {player} player in following sequence : {moves}? Write the move in Standard Algebraic Notation."

    time.sleep(1)

    # retrieve a dictionary encoding the answer
    # function taken from : https://platform.openai.com/docs/guides/chat/introduction
    completions = openai.Completion.create(
        engine=model_engine,
        prompt = prompt,
        max_tokens=400,
        n=1,
        stop=None,
        temperature=0.1,
    )

    # only record the text and discard other information
    message = completions.choices[0].text

    # obtain the chess notation for the next move
    next_move = message.split()[-1]

    # the sentence may end with a dot
    if next_move[-1] == ".":
        next_move = next_move[:-1]

    # the move might also have the number of move as the first character
    # this needs to be removed
    next_move = next_move.split(".")[-1]
    while next_move[0] in ['1', '2', '3', '4', '5', '6', '7', '8']:
        next_move = next_move[1:]

    return next_move


In [4]:
def stockfish_generate_top_best_moves(moves, n = 10):
    """
    Provide the top best candidates for the following move given a sequence of previously
    done chess moves.
    """

    # initialize chess engine
    stockfish = Stockfish(path = os.getcwd() + '\\stockfish_15.1_win_x64_avx2/stockfish-windows-2022-x86-64-avx2.exe')

    # the accuracy of the analysis depends on the depth of the lines studies
    stockfish.set_depth(depth_value=5)

    # measures the strength of the chess engine : a low ELO indicates a weak player
    stockfish.set_elo_rating(elo_rating=2000)

    # define initial configuration
    stockfish.set_position(moves = moves)

    # CASE 1 : employ the function for a top of best moves
    top_best_next_move = stockfish.get_top_moves(num_top_moves=20)

    if n == 1:
        # CASE 2 : use the chess engine to actually perform a move
        top_best_next_move = top_best_next_move[0]['Move']

    return top_best_next_move


In [5]:
def set_chess_board(initial_moves, verbose = False):
    """
    Set the initial configuration of the board. Note that this uses **uci** notation
    (as oposed to ChatGPT prompts for which we use san notation).
    """

    # initialize the board
    board = chess.Board()

    for move in initial_moves:

        m = chess.Move.from_uci(move)

        # stop the game if an illegal move is made
        if m in board.legal_moves:

            board.push(m)

        else:
            print(m)
            raise ValueError("Invalid Move")

    return board

In [6]:
def retrieve_moves_and_scores(stockfish_next_move):
    """
    Given list of next moves generated by stockfish, containing the best moves in Standard Algebraic Notation, 
    together with their scores measured in Centipawns, retrieve the moves and scores separately.

    Parameters
    ----------
    stockfist_next_move : list
        contains dictionaries, each of which encodes the a top next move, toghether with its score measured in Centipawns
    """

    # initialize lists for moves and scores
    move_list = []
    score_list = []

    for m in stockfish_next_move:

        # make sure positions correspond in both lists
        move_list.append(m['Move'])
        score_list.append(m['Centipawn'])

    return move_list, score_list



In [7]:
def convert_uci_to_san(board, uci_move):

    san_move = board.variation_san([chess.Move.from_uci(uci_move)])

    return san_move.split()[-1]

In [8]:
def convert_san_to_uci(board, san_move):

    return board.parse_san(san_move).uci()

In [9]:
def play_game_against_itself(initial_moves_uci, initial_moves_san, max_moves = 1):
    """
    Generate a game of chess played by ChatGPT with itself (at each move, it )
    
    """
    board = set_chess_board(initial_moves_uci)

    # this is a list of strings
    moves_uci = initial_moves_uci.copy()

    # this is a list
    moves_san = initial_moves_san
    
    chatgpt_move_scores = []

    # record the position of the ChatGPT generated move san
    for i in range(max_moves):
        attempts = 0
        print("========================================================")
        print(f"Move {i + 1}")
    
        move_number = len(moves_uci) // 2 + 1

        # one move encompasses one move for white and one for black
        # white starts the game by convention
        for j in range(2):
            if j == 0:
                san_current_move = f" {move_number}."
                player = "White"

            if j == 1:
                san_current_move = f" {move_number}..."
                player = "Black"

            print(f"Player to move : {player}")

            # ask ChatGPT for the best next move
            # this is returned in san format, with no integer indicating the move number

            legal = False
            s = 0
            while not legal and s < 1:
                attempts += 1

                if s == 0:
                    san_chatgpt_next_move = chatgpt_generate_next_move(moves_san, player=player)
                else:
                    san_chatgpt_next_move = chatgpt_generate_next_move(moves_san, player=player, previous_answer=san_chatgpt_next_move, wrong = True)
                try:
                    t = convert_san_to_uci(board, san_chatgpt_next_move)
                    p = chess.Move.from_uci(t)
                    if p in board.legal_moves:
                        legal = True
                        print(f'Attempts : {attempts}')
                except:
                    pass
                    s +=1

            if s == 10:
                return f"Illegal move : {san_chatgpt_next_move}"

            
            print(f"Prompt : {moves_san}")
            print(f'Current ChatGPT Choice : {san_chatgpt_next_move}')

            # update the current move based on ChatGPT prediction
            san_current_move += san_chatgpt_next_move

            # a list of the best next moves generated by a chess engine
            # together with their scores measures in Centipawns
            stockfish_next_move = stockfish_generate_top_best_moves(moves = moves_uci)

            # obtain the moves in uci format, together with their associated scores
            uci_top_next_moves, top_next_moves_scores = retrieve_moves_and_scores(stockfish_next_move)

            # convert the uci moves to san moves so that we can compare them with the ChatGPT answer
            san_top_next_moves = [convert_uci_to_san(board, m) for m in uci_top_next_moves]

            moves_uci.append(convert_san_to_uci(board, san_chatgpt_next_move))

            k = chess.Move.from_uci(moves_uci[-1])

            # update the prompt
            moves_san += san_current_move

            if board.is_check():
                moves_san += "#"

            board.push(k)

            for l, m in enumerate(san_top_next_moves):
            
                if m == san_chatgpt_next_move:
                    chatgpt_move_scores.append(l)

            print(f'Top best moves achieved: {san_chatgpt_next_move in san_top_next_moves}')

            display(board)
            
        print("====================================================")

    return chatgpt_move_scores

In [12]:
def compute_relative_score(board, san_chatgpt_next_move, moves_uci):
    
    stockfish_top_next_moves = stockfish_generate_top_best_moves(moves = moves_uci) 

    # obtain the moves in uci format, together with their associated scores
    uci_top_next_moves, top_next_moves_scores = retrieve_moves_and_scores(stockfish_top_next_moves)

    # convert the uci moves to san moves so that we can compare them with the ChatGPT answer
    uci_chatgpt_next_move = convert_san_to_uci(board, san_chatgpt_next_move)

    x = [-100, -100]
    for l, m in enumerate(uci_top_next_moves):
        #print(m)
        #print(uci_chatgpt_next_move)
        #print("======================================")
        if m == uci_chatgpt_next_move:

            x = [l+1, top_next_moves_scores[l]]

    return x

    

In [15]:
def play_game_against_stockfish_as_white(initial_moves_uci, initial_moves_san, max_moves = 1, verbose = False):
    """
    Generate a game of chess played by ChatGPT with itself (at each move, it )
    
    """
    board = set_chess_board(initial_moves_uci)

    # this is a list of strings
    moves_uci = initial_moves_uci.copy()

    # this is a list
    moves_san = initial_moves_san
    
    chatgpt_move_position = []
    chatgpt_move_scores = []
    try:
        # record the position of the ChatGPT generated move san
        for i in range(max_moves):
                
            if verbose:
                print("========================================================")
                print(f"Move {i + 1}")
            
            move_number = len(moves_uci) // 2 + 1

            # one move encompasses one move for white and one for black
            # white starts the game by convention
            for j in range(2):
                
                if j == 0:

                    if verbose:
                        print("ChatGPT moving")
                    san_current_move = f" {move_number}."
                    player = "White"

                if j == 1:

                    if verbose:
                        print("Stockfish moving")
                    #san_current_move = f" {move_number}..."
                    san_current_move = " "
                    player = "Black"
                
                if verbose:
                    print(f"Player to move : {player}")

                if j == 0:

                    # ask ChatGPT for the best next move
                    # this is returned in san format, with no integer indicating the move number
                    san_chatgpt_next_move = chatgpt_generate_next_move(moves_san, player=player)
                    
                    if verbose:
                        print(f"Prompt : {moves_san}")
                        print(f'Current ChatGPT Choice : {san_chatgpt_next_move}')

                    # update the current move based on ChatGPT prediction
                    san_current_move += san_chatgpt_next_move
                    
                    score = compute_relative_score(board, san_chatgpt_next_move, moves_uci)

                    moves_uci.append(convert_san_to_uci(board, san_chatgpt_next_move))

                    chatgpt_move_position.append(score[0])
                    chatgpt_move_scores.append(score[1])

                    if verbose:
                        print(f"TOP MOVE : {score[0]}")
                        print(f"TOP SCORE : {score[1]}")

                    k = chess.Move.from_uci(moves_uci[-1])

                    # update the prompt
                    moves_san += san_current_move

                    if board.is_check():
                        moves_san += "#"

                    board.push(k)

                if j == 1:
                    
                    stockfish_top_next_moves = stockfish_generate_top_best_moves(moves = moves_uci, n = 10)

                    uci_to_next_moves, _= retrieve_moves_and_scores(stockfish_top_next_moves)

                    stockfish_next_move = np.random.choice(uci_to_next_moves)
                    
                    if verbose:
                        print(f"Stockfish move : {stockfish_next_move}")

                    moves_uci.append(stockfish_next_move)

                    san_current_move += convert_uci_to_san(board, stockfish_next_move)

                    k = chess.Move.from_uci(stockfish_next_move)

                    chatgpt_move_position.append('None')
                    chatgpt_move_scores.append('None')

                    board.push(k)

                    moves_san += san_current_move

                #display(board)

            if verbose:
                print("==============================")

            number_of_moves = i + 1

    except:
        #print("================================================================")
        #display(board)
        #print(san_chatgpt_next_move)
        #print(moves_san)
        pass
    return moves_san, san_chatgpt_next_move, chatgpt_move_position, chatgpt_move_scores, number_of_moves

In [16]:
df = pd.DataFrame()
df['game'] = np.zeros(2000)
df['number of moves'] = np.zeros(2000)
df['faulty_move'] = np.zeros(2000)
df['moves place in top 10'] = np.zeros(2000)
df['moves score (centipawns)'] = np.zeros(2000)


beginning_uci = [["e2e4", "e7e5"], ["e2e4", "c7c6"], ["e2e4", "c7c5"],["e2e4", "g8f6"], ["d2d4", "d7d5"]]
beginning_san = ['1.e4 1...e5', '1.e4 1...c6', '1.e4 1...c5', '1.e4 1...Nf6', '1.d4 1...d5']

k = 0
step = 200
row = 0

for beg_uci, beg_san in zip(beginning_uci, beginning_san):

    for game in range(step):

        print(f"Game {row + 1} started.")
        try:
            moves_san, san_chatgpt_next_move, chatgpt_move_position, chatgpt_move_scores, number_of_moves = \
                play_game_against_stockfish_as_white(initial_moves_uci=beg_uci, initial_moves_san=beg_san, max_moves = 15)
            
            #print(moves_san)
            #print(san_chatgpt_next_move)
            #print(chatgpt_move_position)
            #print(chatgpt_move_scores)
            #print(number_of_moves)
            #print("=============================================")
            df.loc[row, 'game'] = moves_san

            df.loc[row, 'number of moves'] =  number_of_moves

            df.loc[row, 'faulty_move'] = san_chatgpt_next_move

            df.loc[row, 'moves place in top 10'] = str(chatgpt_move_position)

            df.loc[row, 'moves score (centipawns)'] = str(chatgpt_move_scores)

            print(f"Game {row + 1} succeeded.")
            print("========================================")

            row += 1
            #print("succeded")
            #except:
            #    print(2)

        except:
            print(f"Game {row + 1} failed.")
            print("========================================")

    if game == step-1:
        display(df)
display(df)



Game 1 started.
Game 1 succeeded.
Game 2 started.
Game 2 succeeded.
Game 3 started.
Game 3 succeeded.
Game 4 started.
Game 4 succeeded.
Game 5 started.
Game 5 succeeded.
Game 6 started.
Game 6 succeeded.
Game 7 started.
Game 7 succeeded.
Game 8 started.
Game 8 succeeded.
Game 9 started.
Game 9 succeeded.
Game 10 started.
Game 10 succeeded.
Game 11 started.
Game 11 succeeded.
Game 12 started.
Game 12 succeeded.
Game 13 started.
Game 13 succeeded.
Game 14 started.
Game 14 succeeded.
Game 15 started.
Game 15 succeeded.
Game 16 started.
Game 16 succeeded.
Game 17 started.
Game 17 succeeded.
Game 18 started.
Game 18 succeeded.
Game 19 started.
Game 19 succeeded.
Game 20 started.
Game 20 succeeded.
Game 21 started.
Game 21 succeeded.
Game 22 started.
Game 22 succeeded.
Game 23 started.
Game 23 succeeded.
Game 24 started.
Game 24 succeeded.
Game 25 started.
Game 25 succeeded.
Game 26 started.
Game 26 succeeded.
Game 27 started.
Game 27 succeeded.
Game 28 started.
Game 28 succeeded.
Game 29 st

,game,number of moves,faulty_move,moves place in top 10,moves score (centipawns)
0,1.e4 1...e5 2.Nf3 2...Bb4 3.Bd3 3...f6 4.Nc3 4...,3.0,Bxb4,"[2, 'None', 8, 'None', 10, 'None']","[49, 'None', 55, 'None', 48, 'None']"
1,1.e4 1...e5 2.Nf3 2...h6 3.d4 3...b6 4.dxe5 4....,15.0,Bxh6,"[2, 'None', 11, 'None', 2, 'None', -100, 'None...","[49, 'None', 34, 'None', 266, 'None', -100, 'N..."
2,1.e4 1...e5 2.Nf3 2...h6 3.Bc4 3...a5 4.d4 4.....,7.0,Nd5,"[2, 'None', 3, 'None', 4, 'None', 4, 'None', 4...","[49, 'None', 72, 'None', 149, 'None', 423, 'No..."
3,1.e4 1...e5 2.Nf3 2...Bd6 3.Bc4 3...b5 4.Bb3 4...,5.0,Bxc6,"[2, 'None', 6, 'None', 2, 'None', 5, 'None', 2...","[49, 'None', 68, 'None', 149, 'None', 135, 'No..."
4,1.e4 1...e5 2.Nf3 2...h5 3.Nxe5 3...Nh6 4.Nf7 ...,5.0,Qxh4,"[2, 'None', 1, 'None', -100, 'None', 3, 'None'...","[49, 'None', 176, 'None', -100, 'None', 411, '..."
...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0
1996,0.0,0.0,0.0,0.0,0.0
1997,0.0,0.0,0.0,0.0,0.0
1998,0.0,0.0,0.0,0.0,0.0


Game 201 started.
Game 201 succeeded.
Game 202 started.
Game 202 succeeded.
Game 203 started.
Game 203 succeeded.
Game 204 started.
Game 204 succeeded.
Game 205 started.
Game 205 succeeded.
Game 206 started.
Game 206 succeeded.
Game 207 started.
Game 207 succeeded.
Game 208 started.
Game 208 succeeded.
Game 209 started.
Game 209 succeeded.
Game 210 started.
Game 210 succeeded.
Game 211 started.
Game 211 succeeded.
Game 212 started.
Game 212 succeeded.
Game 213 started.
Game 213 succeeded.
Game 214 started.
Game 214 succeeded.
Game 215 started.
Game 215 succeeded.
Game 216 started.
Game 216 succeeded.
Game 217 started.
Game 217 succeeded.
Game 218 started.
Game 218 succeeded.
Game 219 started.
Game 219 succeeded.
Game 220 started.
Game 220 succeeded.
Game 221 started.
Game 221 succeeded.
Game 222 started.
Game 222 succeeded.
Game 223 started.
Game 223 succeeded.
Game 224 started.
Game 224 succeeded.
Game 225 started.
Game 225 succeeded.
Game 226 started.
Game 226 succeeded.
Game 227 sta

,game,number of moves,faulty_move,moves place in top 10,moves score (centipawns)
0,1.e4 1...e5 2.Nf3 2...Bb4 3.Bd3 3...f6 4.Nc3 4...,3.0,Bxb4,"[2, 'None', 8, 'None', 10, 'None']","[49, 'None', 55, 'None', 48, 'None']"
1,1.e4 1...e5 2.Nf3 2...h6 3.d4 3...b6 4.dxe5 4....,15.0,Bxh6,"[2, 'None', 11, 'None', 2, 'None', -100, 'None...","[49, 'None', 34, 'None', 266, 'None', -100, 'N..."
2,1.e4 1...e5 2.Nf3 2...h6 3.Bc4 3...a5 4.d4 4.....,7.0,Nd5,"[2, 'None', 3, 'None', 4, 'None', 4, 'None', 4...","[49, 'None', 72, 'None', 149, 'None', 423, 'No..."
3,1.e4 1...e5 2.Nf3 2...Bd6 3.Bc4 3...b5 4.Bb3 4...,5.0,Bxc6,"[2, 'None', 6, 'None', 2, 'None', 5, 'None', 2...","[49, 'None', 68, 'None', 149, 'None', 135, 'No..."
4,1.e4 1...e5 2.Nf3 2...h5 3.Nxe5 3...Nh6 4.Nf7 ...,5.0,Qxh4,"[2, 'None', 1, 'None', -100, 'None', 3, 'None'...","[49, 'None', 176, 'None', -100, 'None', 411, '..."
...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0
1996,0.0,0.0,0.0,0.0,0.0
1997,0.0,0.0,0.0,0.0,0.0
1998,0.0,0.0,0.0,0.0,0.0


Game 401 started.
Game 401 succeeded.
Game 402 started.
Game 402 succeeded.
Game 403 started.
Game 403 succeeded.
Game 404 started.
Game 404 succeeded.
Game 405 started.
Game 405 succeeded.
Game 406 started.
Game 406 succeeded.
Game 407 started.
Game 407 succeeded.
Game 408 started.
Game 408 succeeded.
Game 409 started.
Game 409 succeeded.
Game 410 started.
Game 410 succeeded.
Game 411 started.
Game 411 succeeded.
Game 412 started.
Game 412 succeeded.
Game 413 started.
Game 413 succeeded.
Game 414 started.
Game 414 succeeded.
Game 415 started.
Game 415 succeeded.
Game 416 started.
Game 416 succeeded.
Game 417 started.
Game 417 succeeded.
Game 418 started.
Game 418 succeeded.
Game 419 started.
Game 419 succeeded.
Game 420 started.
Game 420 succeeded.
Game 421 started.
Game 421 succeeded.
Game 422 started.
Game 422 succeeded.
Game 423 started.
Game 423 succeeded.
Game 424 started.
Game 424 succeeded.
Game 425 started.
Game 425 succeeded.
Game 426 started.
Game 426 succeeded.
Game 427 sta

...

Game 601 started.
Game 601 succeeded.
Game 602 started.
Game 602 succeeded.
Game 603 started.
Game 603 succeeded.
Game 604 started.
Game 604 succeeded.
Game 605 started.
Game 605 succeeded.
Game 606 started.
Game 606 succeeded.
Game 607 started.
Game 607 succeeded.
Game 608 started.
Game 608 succeeded.
Game 609 started.
Game 609 succeeded.
Game 610 started.
Game 610 succeeded.
Game 611 started.
Game 611 succeeded.
Game 612 started.
Game 612 succeeded.
Game 613 started.
Game 613 succeeded.
Game 614 started.
Game 614 succeeded.
Game 615 started.
Game 615 succeeded.
Game 616 started.
Game 616 succeeded.
Game 617 started.
Game 617 succeeded.
Game 618 started.
Game 618 succeeded.
Game 619 started.
Game 619 succeeded.
Game 620 started.
Game 620 succeeded.
Game 621 started.
Game 621 succeeded.
Game 622 started.
Game 622 succeeded.
Game 623 started.
Game 623 succeeded.
Game 624 started.
Game 624 succeeded.
Game 625 started.
Game 625 succeeded.
Game 626 started.
Game 626 succeeded.
Game 627 sta

,game,number of moves,faulty_move,moves place in top 10,moves score (centipawns)
0,1.e4 1...e5 2.Nf3 2...Bb4 3.Bd3 3...f6 4.Nc3 4...,3.0,Bxb4,"[2, 'None', 8, 'None', 10, 'None']","[49, 'None', 55, 'None', 48, 'None']"
1,1.e4 1...e5 2.Nf3 2...h6 3.d4 3...b6 4.dxe5 4....,15.0,Bxh6,"[2, 'None', 11, 'None', 2, 'None', -100, 'None...","[49, 'None', 34, 'None', 266, 'None', -100, 'N..."
2,1.e4 1...e5 2.Nf3 2...h6 3.Bc4 3...a5 4.d4 4.....,7.0,Nd5,"[2, 'None', 3, 'None', 4, 'None', 4, 'None', 4...","[49, 'None', 72, 'None', 149, 'None', 423, 'No..."
3,1.e4 1...e5 2.Nf3 2...Bd6 3.Bc4 3...b5 4.Bb3 4...,5.0,Bxc6,"[2, 'None', 6, 'None', 2, 'None', 5, 'None', 2...","[49, 'None', 68, 'None', 149, 'None', 135, 'No..."
4,1.e4 1...e5 2.Nf3 2...h5 3.Nxe5 3...Nh6 4.Nf7 ...,5.0,Qxh4,"[2, 'None', 1, 'None', -100, 'None', 3, 'None'...","[49, 'None', 176, 'None', -100, 'None', 411, '..."
...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0
1996,0.0,0.0,0.0,0.0,0.0
1997,0.0,0.0,0.0,0.0,0.0
1998,0.0,0.0,0.0,0.0,0.0


Game 801 started.
Game 801 succeeded.
Game 802 started.
Game 802 succeeded.
Game 803 started.
Game 803 succeeded.
Game 804 started.
Game 804 succeeded.
Game 805 started.
Game 805 succeeded.
Game 806 started.
Game 806 succeeded.
Game 807 started.
Game 807 succeeded.
Game 808 started.
Game 808 succeeded.
Game 809 started.
Game 809 succeeded.
Game 810 started.
Game 810 succeeded.
Game 811 started.
Game 811 succeeded.
Game 812 started.
Game 812 succeeded.
Game 813 started.
Game 813 succeeded.
Game 814 started.
Game 814 succeeded.
Game 815 started.
Game 815 succeeded.
Game 816 started.
Game 816 succeeded.
Game 817 started.
Game 817 succeeded.
Game 818 started.
Game 818 succeeded.
Game 819 started.
Game 819 succeeded.
Game 820 started.
Game 820 succeeded.
Game 821 started.
Game 821 succeeded.
Game 822 started.
Game 822 succeeded.
Game 823 started.
Game 823 succeeded.
Game 824 started.
Game 824 succeeded.
Game 825 started.
Game 825 succeeded.
Game 826 started.
Game 826 succeeded.
Game 827 sta

,game,number of moves,faulty_move,moves place in top 10,moves score (centipawns)
0,1.e4 1...e5 2.Nf3 2...Bb4 3.Bd3 3...f6 4.Nc3 4...,3.0,Bxb4,"[2, 'None', 8, 'None', 10, 'None']","[49, 'None', 55, 'None', 48, 'None']"
1,1.e4 1...e5 2.Nf3 2...h6 3.d4 3...b6 4.dxe5 4....,15.0,Bxh6,"[2, 'None', 11, 'None', 2, 'None', -100, 'None...","[49, 'None', 34, 'None', 266, 'None', -100, 'N..."
2,1.e4 1...e5 2.Nf3 2...h6 3.Bc4 3...a5 4.d4 4.....,7.0,Nd5,"[2, 'None', 3, 'None', 4, 'None', 4, 'None', 4...","[49, 'None', 72, 'None', 149, 'None', 423, 'No..."
3,1.e4 1...e5 2.Nf3 2...Bd6 3.Bc4 3...b5 4.Bb3 4...,5.0,Bxc6,"[2, 'None', 6, 'None', 2, 'None', 5, 'None', 2...","[49, 'None', 68, 'None', 149, 'None', 135, 'No..."
4,1.e4 1...e5 2.Nf3 2...h5 3.Nxe5 3...Nh6 4.Nf7 ...,5.0,Qxh4,"[2, 'None', 1, 'None', -100, 'None', 3, 'None'...","[49, 'None', 176, 'None', -100, 'None', 411, '..."
...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0
1996,0.0,0.0,0.0,0.0,0.0
1997,0.0,0.0,0.0,0.0,0.0
1998,0.0,0.0,0.0,0.0,0.0


,game,number of moves,faulty_move,moves place in top 10,moves score (centipawns)
0,1.e4 1...e5 2.Nf3 2...Bb4 3.Bd3 3...f6 4.Nc3 4...,3.0,Bxb4,"[2, 'None', 8, 'None', 10, 'None']","[49, 'None', 55, 'None', 48, 'None']"
1,1.e4 1...e5 2.Nf3 2...h6 3.d4 3...b6 4.dxe5 4....,15.0,Bxh6,"[2, 'None', 11, 'None', 2, 'None', -100, 'None...","[49, 'None', 34, 'None', 266, 'None', -100, 'N..."
2,1.e4 1...e5 2.Nf3 2...h6 3.Bc4 3...a5 4.d4 4.....,7.0,Nd5,"[2, 'None', 3, 'None', 4, 'None', 4, 'None', 4...","[49, 'None', 72, 'None', 149, 'None', 423, 'No..."
3,1.e4 1...e5 2.Nf3 2...Bd6 3.Bc4 3...b5 4.Bb3 4...,5.0,Bxc6,"[2, 'None', 6, 'None', 2, 'None', 5, 'None', 2...","[49, 'None', 68, 'None', 149, 'None', 135, 'No..."
4,1.e4 1...e5 2.Nf3 2...h5 3.Nxe5 3...Nh6 4.Nf7 ...,5.0,Qxh4,"[2, 'None', 1, 'None', -100, 'None', 3, 'None'...","[49, 'None', 176, 'None', -100, 'None', 411, '..."
...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0
1996,0.0,0.0,0.0,0.0,0.0
1997,0.0,0.0,0.0,0.0,0.0
1998,0.0,0.0,0.0,0.0,0.0
